In [1]:
#@title Mount google drive

from google.colab import drive
drive.mount('/content/drive')

%cd './drive/MyDrive/gaze_estimation'

# %cd '../'
# !git clone https://github.com/Selbias/hMeNets.git
# #cd './hMeNets'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/gaze_estimation


In [2]:
import os
import time
import copy
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm.auto import tqdm
from sklearn.linear_model import LinearRegression

In [3]:
#@title Load preprocessed data

# images, head_poses and gazes are already flipped

id_vector = np.load('./data/MPIIGaze_ids.npy')
is_right = np.load('./data/MPIIGaze_is_right.npy')
images = np.load('./data/MPIIGaze_images.npy').reshape(-1,36,60)
head_poses = np.load('./data/MPIIGaze_head_poses.npy')
gazes = np.load('./data/MPIIGaze_gazes.npy')

In [4]:
#@title data balancing

indice_list = []
np.random.seed(42)
for i in tqdm(range(15)) :
    if i < 10 :
        left_indice = np.random.choice(np.where((id_vector == f"p0{i}") & np.invert(is_right))[0], size=1500, replace=False)
        right_indice = np.random.choice(np.where((id_vector == f"p0{i}") & is_right)[0], size=1500, replace=False)
        total_indice = np.concatenate([left_indice, right_indice])
    elif i != 13 :
        left_indice = np.random.choice(np.where((id_vector == f"p{i}") & np.invert(is_right))[0], size=1500, replace=False)
        right_indice = np.random.choice(np.where((id_vector == f"p{i}") & is_right)[0], size=1500, replace=False)
        total_indice = np.concatenate([left_indice, right_indice])
    else :
        left_indice = np.random.choice(np.where((id_vector == f"p{i}") & np.invert(is_right))[0], size=2, replace=False)
        right_indice = np.random.choice(np.where((id_vector == f"p{i}") & is_right)[0], size=2, replace=False)
        total_indice = np.concatenate([np.where(id_vector == f"p{i}")[0], left_indice, right_indice])
    indice_list.append(total_indice)

selected_ids = [id_vector[indice] for indice in indice_list]
selected_images = [torch.as_tensor(images[indice]).float() for indice in indice_list]
selected_head_poses = [torch.as_tensor(head_poses[indice]).float() for indice in indice_list]
selected_gazes = [torch.as_tensor(gazes[indice]).float() for indice in indice_list]

  0%|          | 0/15 [00:00<?, ?it/s]

In [5]:

from loss.likelihood_hetero import multivariate_nll_hetero_i, multivariate_njll_hetero_i
from hglm.covariance_module import covariance_module
from loss.h_likelihood_covariance import nhll_hetero_arbitrary
from util import make_reproducibility, TensorDataset, convert_to_xyz, mae, make_arbitrary_masking, k_fold_index
from networks import *
from hglm.hglm_hetero_arbitrary import hetero_covariance_without_val

/content/drive/MyDrive/gaze_estimation/PyTorch
/content/drive/MyDrive/gaze_estimation


# Data Load

In [6]:
# #@title test
# total_ids = np.concatenate(selected_ids)
# total_images = torch.cat(selected_images)
# total_hps = torch.cat(selected_head_poses)
# total_gazes = torch.cat(selected_gazes)

# total_N = len(total_ids)
# num_fold=10
# cv_seed = 100
# cv_indice_list = [k_fold_index(N=3000, k=num_fold, randomize=True, SEED=cv_seed + i) for i in range(15)]
# total_ids_unique = np.unique(total_ids)

# from loss.likelihood_hetero import multivariate_nll_hetero_i, multivariate_njll_hetero_i
# from hglm.covariance_module import covariance_module
# from loss.h_likelihood_covariance import nhll_hetero_arbitrary
# from hglm.hglm_hetero_arbitrary import hetero_covariance_without_val

# for fold in [0] :
#     train_indice = np.concatenate([np.where(total_ids == total_ids_unique[i])[0][cv_indice_list[i][fold][0]] for i in range(15)])
#     test_indice = np.concatenate([np.where(total_ids == total_ids_unique[i])[0][cv_indice_list[i][fold][1]] for i in range(15)])
#     print(test_indice[:5])

#     train_ids = total_ids[train_indice]
#     train_images = total_images[train_indice].float()
#     train_hps = total_hps[train_indice].float()
#     train_gazes = total_gazes[train_indice].float()

#     test_ids = total_ids[test_indice]
#     test_images = total_images[test_indice].float()
#     test_hps = total_hps[test_indice].float()
#     test_gazes = total_gazes[test_indice].float()

#     hetero_covariance_without_val(
#         train_ids, train_images, train_hps, train_gazes,
#         test_ids, test_images, test_hps, test_gazes,
#         ResNet_batchnorm.ResNet_batchnorm, hidden_features=500, K=2,
#         mean_lr=5e-3, variance_lr=1e-3, weight_decay=0, batch_size=1000,
#         pretrain_iter=1, m_pretrain_epoch=50, v_pretrain_epoch=1, max_iter=1, mean_epoch=1, v_step_iter=1, patience=10,
#         device=torch.device('cuda:0'), experiment_name='test', SEED=10,
#         normalize=True, deg=True, test_unseen=False, weighted=True, variance_check=True, verbose=False, bins=50, reset_opt=False)


In [7]:
total_ids = np.concatenate(selected_ids)
total_images = torch.cat(selected_images)
total_hps = torch.cat(selected_head_poses)
total_gazes = torch.cat(selected_gazes)

total_N = len(total_ids)
num_fold=10

In [11]:
device = torch.device('cuda:0')
seed = 10

experiment_name = 'cov_within'
batch_size = 1000
pretrain_iter = 1
m_pretrain_epoch = 50
v_pretrain_epoch = 20
max_iter = 150
mean_epoch = 10
v_step_iter = 100
patience = 10
mean_lr = 5e-3
variance_lr = 1e-3
weight_decay = 0
hidden_features = 500
reset_opt=False
initialize_Sigma=True

In [9]:
cv_seed = 100
cv_indice_list = [k_fold_index(N=3000, k=num_fold, randomize=True, SEED=cv_seed + i) for i in range(15)]
total_ids_unique = np.unique(total_ids)

res_list = []

In [ ]:
#@title test

for fold in [0] :
    train_indice = np.concatenate([np.where(total_ids == total_ids_unique[i])[0][cv_indice_list[i][fold][0]] for i in range(15)])
    test_indice = np.concatenate([np.where(total_ids == total_ids_unique[i])[0][cv_indice_list[i][fold][1]] for i in range(15)])
    print(test_indice[:5])

    train_ids = total_ids[train_indice]
    train_images = total_images[train_indice].float()
    train_hps = total_hps[train_indice].float()
    train_gazes = total_gazes[train_indice].float()

    test_ids = total_ids[test_indice]
    test_images = total_images[test_indice].float()
    test_hps = total_hps[test_indice].float()
    test_gazes = total_gazes[test_indice].float()

    res_list.append(hetero_covariance_without_val(
        train_ids, train_images, train_hps, train_gazes,
        test_ids, test_images, test_hps, test_gazes,
        ResNet_batchnorm.ResNet_batchnorm, hidden_features=hidden_features, K=2, initialize_Sigma=initialize_Sigma,
        mean_lr=mean_lr, variance_lr=variance_lr, weight_decay=weight_decay, batch_size=batch_size,
        pretrain_iter=pretrain_iter, m_pretrain_epoch=m_pretrain_epoch, v_pretrain_epoch=v_pretrain_epoch, max_iter=max_iter, mean_epoch=mean_epoch, v_step_iter=v_step_iter, patience=patience,
        device=device, experiment_name=f'{experiment_name}_{fold}', SEED=seed + fold,
        normalize=True, deg=True, test_unseen=False, weighted=True, variance_check=True, verbose=False, bins=50, reset_opt=reset_opt))




Output hidden; open in https://colab.research.google.com to view.